In [1]:
import pandas as pd
import numpy as np

# Problem 2: a linguistic street map of Singapore

There is considerable linguistic variety in Singapore road names: Malaysians names (Jalan Besar), British names (Northumberland road), Chinese names (Keong Saik Road), Indian names (Veerasamy road), Jewish names (Belilios road), and the usual generic sorts of names that describe either area landmarks or other common noun.

![Image](streets.png)

In [2]:
# load the dataset
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/streets.csv'
street_names = pd.read_csv(url)
street_names.head(10)

,name,tag,origin
0,Saiboo,Street,Indian
1,Merchant,Loop,Generic
2,Hill,Street,Generic
3,Ophir,Road,Malay
4,Buona,Vista Road,Other
5,Sengkang,Avenue,Chinese
6,Ang Mo Kio,Avenue,Chinese
7,Brickland,Road,British
8,Choa Chu Kang,Road,Chinese
9,Woodlands,Avenue,Generic


**``Origin`` column values:**

- Chinese (all dialects including Cantonese, Hokkien, Mandarin, etc)
- Malay
- Indian (all languages of the subcontinent)
- British
- Generic (Race Course Road, Sunrise Place, etc)
- Other (Other languages).

In [3]:
street_names.origin.value_counts()

Malay      927
British    798
Generic    497
Chinese    403
Other      167
Indian      42
Name: origin, dtype: int64

The **goal** is to predict the ``origin`` column from the ``street`` and ``tag`` columns.

**Part 1:** Add the following features:

- Number of words in road name (more words => more likely to be Chinese.) Assign it to a column named `n_words`.
- Average word length in road name (longer words => more likely to be British or Indian.) Assign it to a column named `avg_word_len`.
- Is the road tag Malay? (if yes => very correlated with being Malay.) **Malay tags**: *Jalan*, *Lorong*, *Bukit*, *Lengkok*, *Taman*, *Kampong*, *Lengkong*. Assign it to a column named `is_malay`.

In [4]:
street_names['n_words'] = street_names['name'].apply(lambda x: len(x.split()))
street_names['avg_word_len'] = street_names['name'].apply(lambda x: np.mean([len(word) for word in x.split()]))
street_names['is_malay'] = street_names['name'].apply(lambda x: 1 if any(word in x for word in ['Jalan', 'Lorong', 'Bukit', 'Lengkok', 'Taman', 'Kampong', 'Lengkong']) else 0)

In [5]:
street_names

,name,tag,origin,n_words,avg_word_len,is_malay
0,Saiboo,Street,Indian,1,6.0,0
1,Merchant,Loop,Generic,1,8.0,0
2,Hill,Street,Generic,1,4.0,0
3,Ophir,Road,Malay,1,5.0,0
4,Buona,Vista Road,Other,1,5.0,0
...,...,...,...,...,...,...
2829,Florence,Close,Chinese,1,8.0,0
2830,Hoot Kiam,Road,Chinese,2,4.0,0
2831,Clarke,Quay,British,1,6.0,0
2832,Countryside,Walk,Generic,1,11.0,0


**Part 2:** define the matrix X (columns `name`, `n_words`, `avg_word_len`, `is_malay`) and the target vector y (column `origin`)

In [6]:
X = street_names[['name', 'n_words', 'avg_word_len', 'is_malay']]
y = street_names['origin']

**Part 3:** split X and y into training and testing sets.

In [7]:
# split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

**Part 4:** build a classification pipeline.

The pipeline must use `CountVectorizer` to compute the number of **character 1-gram, 2-grams, 3-grams and 4-grams** from the column `name` ([CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) has an `analyzer` parameter that you'll have to set to `analyzer='char'`.)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([('vectorizer', CountVectorizer(analyzer='char', ngram_range=(1,4))), ('scaler', StandardScaler(with_mean=False)), ('classifier', LogisticRegression(max_iter=10000))])

**Part 5:** Fit the classification pipeline to the training data, and  evaluate its performance on the testing set.

In [11]:
pipe.fit(X_train['name'], y_train)
pipe.score(X_test['name'], y_test)

0.7940761636107193